In [0]:
catalogo = "medalhao"
schema_gold  = "gold"

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window

df_pedido_total_silver = spark.table("medalhao.silver.pedido_total_silver")
df_consumidores_silver = spark.table("medalhao.silver.consumidores_silver")
df_pedidos_silver = spark.table("medalhao.silver.pedidos_silver")
df_itens_pedidos_silver = spark.table("medalhao.silver.pedidos_itens_silver")
df_produtos_silver = spark.table("medalhao.silver.produtos_silver")
df_vendedores_silver = spark.table("medalhao.silver.vendedores_silver")
df_avaliacao_silver = spark.table("medalhao.silver.avaliacoes_pedidos_validos_silver")
df_cotacao_silver = spark.table("medalhao.silver.cotacao_dolar_silver")

In [0]:
ft_vendas_consumidor_local = (df_pedido_total_silver
                                .join
                                (df_consumidores_silver, on="id_consumidor")
                                .select
                                (
                                    F.col("id_consumidor"),
                                    F.col("id_pedido"),
                                    F.col("data_pedido"),
                                    F.col("valor_total_pago_brl").alias("valor_total_pedido_brl"),
                                    F.col("estado"),
                                    F.col("cidade"),
                                    F.col("data_pedido")
                                )
                            )

ft_vendas_consumidor_local.printSchema()

In [0]:
df_total_compras_por_consumidor = (ft_vendas_consumidor_local.
                                   groupBy("cidade", "estado")
                                   .agg(
                                        F.sum("valor_total_pedido_brl").alias("valor_total_localidade"),
                                        F.count("id_pedido").alias("quantidade_vendas")
                                   ))


display(df_total_compras_por_consumidor.limit(10))

df_total_compras_por_consumidor.createOrReplaceTempView("view_total_compras_por_consumidor")

In [0]:
df_total_vendas_por_estado = spark.sql("SELECT estado, COUNT(quantidade_vendas) FROM view_total_compras_por_consumidor GROUP BY estado")
display(df_total_vendas_por_estado.limit(10))

In [0]:
df_consumidor_pedidos_itens = (
    df_consumidores_silver
    .join(df_pedidos_silver, on="id_consumidor")
    .join(df_itens_pedidos_silver, on="id_pedido")
    .select(
        F.col("id_pedido"),
        F.col("id_consumidor"),
        F.col("id_vendedor"),
        F.col("entrega_no_prazo"),
        F.col("tempo_entrega_dias"),
        F.col("tempo_entrega_estimado_dias"),
        F.col("cidade"),
        F.col("estado")
    )
)


In [0]:
df_tempo_medio_entrega_localidade = (df_consumidor_pedidos_itens.
                                     filter(F.col("entrega_no_prazo") != "Não entregue").
                                     groupBy("cidade", "estado").
                                      agg(
                                          F.avg("tempo_entrega_dias").alias("tempo_medio_entrega"),
                                          F.avg("tempo_entrega_estimado_dias").alias("tempo_medio_entrega_estimado")
                                      )
                                      .withColumn("entrega_maior_que_estimado", F.when(F.col("tempo_medio_entrega") > F.col("tempo_medio_entrega_estimado"), "Sim").otherwise("Não")))
df_tempo_medio_entrega_localidade.createOrReplaceTempView("view_tempo_medio_entrega_localidade")

In [0]:
df_vendedor_pontualidade = (df_consumidor_pedidos_itens
                            .groupBy("id_vendedor")
                            .agg(
                                F.count(F.when(F.col("entrega_no_prazo") == "Não", F.col("entrega_no_prazo"))).alias("total_atrasados"),
                                F.count(F.col("id_pedido")).alias("total_pedidos")
                            )
                            .withColumn("percentual_atraso", (F.col("total_atrasados") * 100 / F.col("total_pedidos")))
                            .withColumn("percentual_atraso", F.concat(F.col("percentual_atraso").cast("string"), F.lit("%")))
                            )
df_vendedor_pontualidade.createOrReplaceTempView("view_df_vendedor_pontualidade")

In [0]:
from pyspark.sql import functions as F

data_inicio = dbutils.widgets.get("data_inicio")
data_fim = dbutils.widgets.get("data_fim")

df = spark.sql(
    f"SELECT DATE('{data_inicio}') AS data_inicio, DATE('{data_fim}') AS data_fim"
)

dm_tempo = (
    df
    .withColumn("array_de_dias_a_adicionar", F.sequence(F.lit(1), F.date_diff("data_fim", "data_inicio")))
    .withColumn("dias_a_adicionar", F.explode("array_de_dias_a_adicionar"))
    .withColumn("sk_tempo", F.date_add("data_inicio", "dias_a_adicionar"))
    .withColumn("ano", F.year("sk_tempo"))
    .withColumn("trimestre", F.quarter("sk_tempo"))
    .withColumn("mes", F.month("sk_tempo"))
    .withColumn("semana_do_ano", F.weekofyear("sk_tempo"))
    .withColumn("dia", F.dayofmonth("sk_tempo"))
    .withColumn("dia_da_semana_num", F.dayofweek("sk_tempo"))
    .withColumn("dia_da_semana_nome", F.expr("""
        CASE dayofweek(sk_tempo)
            WHEN 1 THEN 'Domingo'
            WHEN 2 THEN 'Segunda-feira'
            WHEN 3 THEN 'Terça-feira'
            WHEN 4 THEN 'Quarta-feira'
            WHEN 5 THEN 'Quinta-feira'
            WHEN 6 THEN 'Sexta-feira'
            WHEN 7 THEN 'Sábado'
        END
    """))
    .withColumn("mes_nome", F.expr("""
        CASE month(sk_tempo)
            WHEN 1 THEN 'Janeiro'
            WHEN 2 THEN 'Fevereiro'
            WHEN 3 THEN 'Março'
            WHEN 4 THEN 'Abril'
            WHEN 5 THEN 'Maio'
            WHEN 6 THEN 'Junho'
            WHEN 7 THEN 'Julho'
            WHEN 8 THEN 'Agosto'
            WHEN 9 THEN 'Setembro'
            WHEN 10 THEN 'Outubro'
            WHEN 11 THEN 'Novembro'
            WHEN 12 THEN 'Dezembro'
        END
    """))
    .withColumn("eh_fim_de_semana", F.when((F.col("dia_da_semana_num") == 1) | (F.col("dia_da_semana_num") == 7), "Sim").otherwise("Não"))
    .drop("dias_a_adicionar", "data_fim", "array_de_dias_a_adicionar", "data_inicio")
)

dm_tempo.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{catalogo}.{schema_gold}.dm_tempo")

In [0]:
dim_clientes_df = (df_consumidores_silver
                   .select(
                       F.col("id_consumidor").alias("id_cliente"),
                       F.col("cidade"),
                       F.col("prefixo_cep"),
                       F.col("estado")
                   ))

dim_produtos_df = (df_produtos_silver
                   .select(
                       F.col("id_produto").alias("id_produto"),
                       F.col("categoria_produto"),
                       F.col("comprimento_centimetros"),
                       F.col("altura_centimetros"),
                       F.col("largura_centimetros"),
                       F.col("peso_produto_gramas"),
                   ))

dim_produtos_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{catalogo}.{schema_gold}.dim_produtos")

dim_vendedores_df = (df_vendedores_silver
                     .select(
                         F.col("id_vendedor").alias("id_vendedor"),),
                         F.col("cidade"),
                         F.col("estado"),
                        F.col("prefixo_cep"),
                     )

In [0]:
dim_media_avaliacao = (
    df_pedido_total_silver
    .join(df_avaliacao_silver, on="id_pedido")
    .groupBy("id_pedido")
    .agg(
        F.avg("avaliacao").alias("media_avaliacao_pedido")
    )
)

display(dim_media_avaliacao.limit(10))

In [0]:
df_itens_pedidos_silver_alias = df_itens_pedidos_silver.alias("itens")
dim_media_avaliacao_alias = dim_media_avaliacao.alias("avaliacao")
df_pedidos_silver_alias = df_pedidos_silver.alias("pedidos")
df_pedido_total_silver_alias = df_pedido_total_silver.alias("pedido_total")
df_cotacao_silver_alias = df_cotacao_silver.alias("cotacao")

ft_vendas_geral = (
    df_itens_pedidos_silver_alias
    .join(dim_media_avaliacao_alias, df_itens_pedidos_silver_alias.id_pedido == dim_media_avaliacao_alias.id_pedido)
    .join(df_pedidos_silver_alias, df_itens_pedidos_silver_alias.id_pedido == df_pedidos_silver_alias.id_pedido)
    .join(df_pedido_total_silver_alias, df_itens_pedidos_silver_alias.id_pedido == df_pedido_total_silver_alias.id_pedido)
    .join(df_cotacao_silver_alias, F.to_date(df_pedido_total_silver_alias.data_pedido) == F.to_date(df_cotacao_silver_alias.dataHoraCotacao))
    .select(
        F.col("itens.id_pedido"),
        F.col("itens.id_item"),
        F.col("pedido_total.id_consumidor").alias("fk_cliente"),
        F.col("itens.id_produto").alias("fk_produto"),
        F.col("itens.id_vendedor").alias("fk_vendedor"),
        F.col("pedido_total.data_pedido").alias("fk_tempo"),
        F.col("pedidos.status").alias("status_pedido"),
        F.col("pedidos.tempo_entrega_dias"),
        F.col("pedidos.entrega_no_prazo"),
        F.col("preco_BRL").alias("valor_produto_brl"),
        F.col("preco_frete").alias("valor_frete_brl"),
        F.col("cotacao.cotacaoCompra").alias("cotacao_dolar"),
        F.col("avaliacao.media_avaliacao_pedido").alias("avaliacao_pedido")
    )
    .withColumn("valor_total_item_brl", F.col("valor_produto_brl") + F.col("valor_frete_brl"))
    .withColumn("valor_produto_usd", F.col("valor_frete_brl") / F.col("cotacao_dolar"))
    .withColumn("valor_frete_usd", F.col("valor_frete_brl") / F.col("cotacao_dolar"))
    .withColumn("valor_total_item_usd", F.col("valor_produto_usd") + F.col("valor_frete_usd"))
)

ft_vendas_geral.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{catalogo}.{schema_gold}.ft_vendas_geral")

In [0]:
df_vendas_por_periodo = (
    ft_vendas_geral
    .join(
        dm_tempo,
        ft_vendas_geral["fk_tempo"] == dm_tempo["sk_tempo"]
    )
    .select(
        dm_tempo["ano"],
        dm_tempo["mes"],
        dm_tempo["trimestre"],
        dm_tempo["semana_do_ano"],
        dm_tempo["dia"],
        dm_tempo["dia_da_semana_num"],
        dm_tempo["dia_da_semana_nome"],
        dm_tempo["mes_nome"],
        dm_tempo["eh_fim_de_semana"],
        ft_vendas_geral["id_pedido"],
        ft_vendas_geral["id_item"],
        ft_vendas_geral["fk_cliente"],
        ft_vendas_geral["fk_produto"],
        ft_vendas_geral["fk_vendedor"],
        ft_vendas_geral["fk_tempo"],
        ft_vendas_geral["status_pedido"],
        ft_vendas_geral["tempo_entrega_dias"],
        ft_vendas_geral["entrega_no_prazo"],
        ft_vendas_geral["valor_produto_brl"],
        ft_vendas_geral["valor_frete_brl"],
        ft_vendas_geral["cotacao_dolar"],
        ft_vendas_geral["avaliacao_pedido"],
        ft_vendas_geral["valor_total_item_brl"],
        ft_vendas_geral["valor_produto_usd"],
        ft_vendas_geral["valor_frete_usd"],
        ft_vendas_geral["valor_total_item_usd"]
    )
)

df_vendas_por_periodo.createOrReplaceTempView("view_vendas_por_periodo_detalhado")

In [0]:
df_vendas_por_periodo = (
    ft_vendas_geral
    .join(
        dm_tempo,
        ft_vendas_geral["fk_tempo"] == dm_tempo["sk_tempo"]
    )
    .groupBy(
        dm_tempo["ano"],
        dm_tempo["mes"],
        dm_tempo["trimestre"],
        dm_tempo["semana_do_ano"],
        dm_tempo["dia"],
        dm_tempo["dia_da_semana_num"],
        dm_tempo["dia_da_semana_nome"],
        dm_tempo["mes_nome"],
        dm_tempo["eh_fim_de_semana"],
    )
    .agg(
        F.countDistinct("id_pedido").alias("total_pedidos"),
        F.count("id_item").alias("total_itens"),
        F.sum("valor_total_item_brl").alias("receita_total_brl"),
        F.sum("valor_total_item_usd").alias("receita_total_usd"),
        (F.sum("valor_total_item_brl") / F.countDistinct("id_pedido")).alias("ticket_medio_brl"),
        F.avg("avaliacao_pedido").alias("avaliacao_media")
    )
)

df_vendas_por_periodo.createOrReplaceTempView("view_vendas_por_periodo")



In [0]:
%sql
SELECT mes_nome FROM view_vendas_por_periodo WHERE avaliacao_media IN (
  SELECT MAX(avaliacao_media) FROM view_vendas_por_periodo
);

SELECT dia_da_semana_nome FROM view_vendas_por_periodo WHERE receita_total_brl IN (
  SELECT MAX(receita_total_brl) FROM view_vendas_por_periodo
);

In [0]:
df_top_produto = (dim_produtos_df.
                  join(
                      ft_vendas_geral,
                      dim_produtos_df["id_produto"] == ft_vendas_geral["fk_produto"]
                  )
                  .groupBy(
                      dim_produtos_df["id_produto"],
                      dim_produtos_df["categoria_produto"],
                  )
                  .agg(
                        F.count(
                            "id_produto"
                        ).alias("quantidade_vendida"),
                        F.count("id_pedido").alias('total_pedidos'),
                        F.avg("peso_produto_gramas").alias("peso_medio_gramas"),
                        F.sum("valor_total_item_brl").alias("receita_brl"),
                        F.sum("valor_total_item_usd").alias("receita_usd"),
                        F.avg("valor_produto_brl").alias("preco_medio_brl"),
                        F.avg("avaliacao_pedido").alias("avaliaco_media")
                  ))

df_top_produto.createOrReplaceTempView("view_top_produto")

In [0]:
spark.sql("USE CATALOG medalhao")
spark.sql("USE SCHEMA gold")

view_vendas_produtos_esteticos = spark.sql(
    """
    WITH view_vendas_produtos_esteticos AS (
        SELECT 
            ano, 
            mes, 
            categoria_produto, 
            COUNT(id_produto) AS total_pedidos, 
            COUNT(id_item) AS total_itens_vendidos,
            SUM(valor_total_item_brl) AS receita_total_brl,
            SUM(valor_produto_usd) AS receita_total_usd,
            receita_total_brl/total_itens_vendidos AS ticket_medio_brl,
            receita_total_usd/total_itens_vendidos AS ticket_medio_usd,
            AVG(avaliacao_pedido) AS avaliacao_media
        FROM ft_vendas_geral v 
        JOIN dim_produtos p ON fk_produto = p.id_produto
        JOIN dm_tempo t ON fk_tempo = t.sk_tempo 
        GROUP BY ano, mes, categoria_produto
        HAVING categoria_produto LIKE 'fashion%'
    )
    SELECT * FROM view_vendas_produtos_esteticos;
    """
)
display(view_vendas_produtos_esteticos
)

In [0]:
df = spark.sql(
    """
    WITH view_vendas_produtos_esteticos AS (
        SELECT 
            ano, 
            mes, 
            categoria_produto, 
            COUNT(id_produto) AS total_pedidos, 
            COUNT(id_item) AS total_itens_vendidos,
            SUM(valor_total_item_brl) AS receita_total_brl,
            SUM(valor_produto_usd) AS receita_total_usd,
            SUM(receita_total_brl/total_itens_vendidos) AS ticket_medio_brl,
            SUM(receita_total_usd/total_itens_vendidos) AS ticket_medio_usd,
            AVG(avaliacao_pedido) AS avaliacao_media
        FROM view_top_produto p
        JOIN view_vendas_por_periodo v 
        ON 
    )
    """
)